In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 0
TIME_RETRIEVAL_TEE_WASM = 13

In [ ]:
def compute_mean_std(name, time):
    df = pd.DataFrame(columns = ["Name","Time"])
    df["Name"] = pd.to_numeric(name)
    df["Time"] = time
    
    result = pd.DataFrame(columns = ["Mean","Std"])
    result["Mean"] = df.groupby("Name")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Name")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    
    return result

def formatForLatex(n, digits_round = 3):
    roundedAsString = str(round(n, digits_round))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
#
## TEE (Wasm) dataset
#

In [ ]:
# Import the dataset
tee_wasm_raw = pd.read_csv("../logs/speedtest1/tee-wasm.csv", names = ["Name","Time"])
tee_wasm_raw["Name"] = tee_wasm_raw["Name"]

tee_wasm_raw

In [ ]:
# Adapt the time according the retrieval of the time
tee_wasm = tee_wasm_raw

tee_wasm["Time"] = tee_wasm["Time"] - 2*TIME_RETRIEVAL_TEE_WASM/2

# Drop any error in the logs
tee_wasm.drop(tee_wasm[tee_wasm.Name.str.contains("failed")].index, inplace = True)
tee_wasm = tee_wasm.reset_index()

tee_wasm

In [ ]:
#
## TEE (Native) dataset
#

In [ ]:
# Import the dataset
tee_native_raw = pd.read_csv("../logs/speedtest1/tee-native.csv", names = ["Name","Time"])
tee_native_raw["Name"] = tee_native_raw["Name"]

# Trim to results to match TEE logs, which may have been trimmed because of errors
tee_native_raw.drop(tee_native_raw.index[len(tee_wasm.index):], inplace = True)

tee_native_raw

In [ ]:
# Adapt the time according the retrieval of the time
tee_native = tee_native_raw

tee_native["Time"] = tee_native["Time"] - 2*TIME_RETRIEVAL_TEE/2

tee_native

In [ ]:
#
## REE (Native) dataset
#

In [ ]:
# Import the dataset
ree_native_raw = pd.read_csv("../logs/speedtest1/ree-native.csv", names = ["Name","Time"])
ree_native_raw["Name"] = ree_native_raw["Name"]

# Trim to results to match TEE logs, which may have been trimmed because of errors
ree_native_raw.drop(ree_native_raw.index[len(tee_wasm.index):], inplace = True)

ree_native_raw

In [ ]:
# Adapt the time according the retrieval of the time
ree_native = ree_native_raw

ree_native["Time"] = ree_native["Time"] - 2*TIME_RETRIEVAL_REE/2

ree_native

In [ ]:
#
## REE (Wasm) dataset
#

In [ ]:
# Import the dataset
ree_wasm_raw = pd.read_csv("../logs/speedtest1/ree-wasm.csv", names = ["Name","Time"])
ree_wasm_raw["Name"] = ree_wasm_raw["Name"]

# Trim to results to match TEE logs, which may have been trimmed because of errors
ree_wasm_raw.drop(ree_wasm_raw.index[len(tee_wasm.index):], inplace = True)

ree_wasm_raw

In [ ]:
# Adapt the time according the retrieval of the time
ree_wasm = ree_wasm_raw

ree_wasm["Time"] = ree_wasm["Time"] - 2*TIME_RETRIEVAL_REE_WASM/2

ree_wasm

In [ ]:
#
## Export
#

In [ ]:
# Normalize the results based on REE time
ree_native_normalized = pd.DataFrame()
ree_wasm_normalized = pd.DataFrame()
tee_native_normalized = pd.DataFrame()
tee_wasm_normalized = pd.DataFrame()
for col in ree_native:
    if col == "Name":
        tee_wasm_normalized[col] = tee_wasm[col]
        ree_wasm_normalized[col] = ree_wasm[col]
        tee_native_normalized[col] = tee_native[col]
        ree_native_normalized[col] = ree_native[col]
    else:
        tee_wasm_normalized[col] = tee_wasm[col] / ree_native[col]
        ree_wasm_normalized[col] = ree_wasm[col] / ree_native[col]
        tee_native_normalized[col] = tee_native[col] / ree_native[col]
        ree_native_normalized[col] = 1
    
tee_wasm_normalized

In [ ]:
# Compute the mean and std
ree_native_export = pd.DataFrame()

ree_native_export = ree_native_export.append(compute_mean_std(ree_native_normalized["Name"], ree_native_normalized["Time"]))
ree_native_export

In [ ]:
# Compute the mean and std
ree_wasm_export = pd.DataFrame()

ree_wasm_export = ree_wasm_export.append(compute_mean_std(ree_wasm_normalized["Name"], ree_wasm_normalized["Time"]))
ree_wasm_export

In [ ]:
# Compute the mean and std
tee_wasm_export = pd.DataFrame()

tee_wasm_export = tee_wasm_export.append(compute_mean_std(tee_wasm_normalized["Name"], tee_wasm_normalized["Time"]))
tee_wasm_export

In [ ]:
# Compute the mean and std
tee_native_export = pd.DataFrame()

tee_native_export = tee_native_export.append(compute_mean_std(tee_native_normalized["Name"], tee_native_normalized["Time"]))
tee_native_export

In [ ]:
#
## Temp visualisation
#

In [ ]:
df = pd.DataFrame({'REE native': ree_native_export["Mean"],
                   'REE Wasm': ree_wasm_export["Mean"],
                   'TEE Wasm': tee_wasm_export["Mean"],
                   'TEE Native': tee_native_export["Mean"]}, index=ree_native_export.index.values)
ax = df.plot.bar()
ax.set_ylim(0.9,3)

In [ ]:
# Retrieve the name of the experiment, order by TEE normalized run time
tee_wasm_indexes = tee_wasm_export.sort_values(by=["Mean"]).index
tee_wasm_indexes

In [ ]:
# Order the entries based on the TEE normalized run time
ree_wasm_export = ree_wasm_export.reindex(tee_wasm_indexes)
tee_wasm_export = tee_wasm_export.reindex(tee_wasm_indexes)
tee_native_export = tee_native_export.reindex(tee_wasm_indexes)

In [ ]:
# Compute values for the paper

def get_mean_of_experiment(name):
    return tee_wasm_export.loc[tee_wasm_export.index == name]["Mean"].values[0]

tee_wasm_mean_of_mean = tee_wasm_export["Mean"].mean()
ree_wasm_mean_of_mean = ree_wasm_export["Mean"].mean()
tee_native_mean_of_mean = tee_native_export["Mean"].mean()
tee_wasm_and_tee_native_ratio = tee_wasm_mean_of_mean / tee_native_mean_of_mean

tee_wasm_insert_mean = pd.DataFrame(
[
    get_mean_of_experiment(100),
    get_mean_of_experiment(110),
    get_mean_of_experiment(120),
    get_mean_of_experiment(180),
    get_mean_of_experiment(190),
    get_mean_of_experiment(210),
    get_mean_of_experiment(290),
    get_mean_of_experiment(300),
    get_mean_of_experiment(400),
    get_mean_of_experiment(500)
]).mean()

tee_wasm_index_mean = pd.DataFrame(
[
    get_mean_of_experiment(110),
    get_mean_of_experiment(120),
    get_mean_of_experiment(160),
    get_mean_of_experiment(161),
    get_mean_of_experiment(170),
    get_mean_of_experiment(180),
    get_mean_of_experiment(230),
    get_mean_of_experiment(270),
    get_mean_of_experiment(400),
    get_mean_of_experiment(410),
    get_mean_of_experiment(500),
    get_mean_of_experiment(510)
]).mean()

tee_wasm_select_mean = pd.DataFrame(
[
    get_mean_of_experiment(130),
    get_mean_of_experiment(140),
    get_mean_of_experiment(142),
    get_mean_of_experiment(145),
    get_mean_of_experiment(160),
    get_mean_of_experiment(161),
    get_mean_of_experiment(170),
    get_mean_of_experiment(260),
    get_mean_of_experiment(310),
    get_mean_of_experiment(320),
    get_mean_of_experiment(410),
    get_mean_of_experiment(510),
    get_mean_of_experiment(520)
]).mean()
tee_wasm_select_mean.values[0]

In [ ]:
ree_wasm_mean_of_mean

In [ ]:
tee_wasm_mean_of_mean

In [ ]:
tee_native_mean_of_mean

In [ ]:
tee_wasm_insert_mean

In [ ]:
tee_wasm_select_mean

In [ ]:
tee_wasm_write_mean

In [ ]:
# Export CSV
#
# File format: experiment_name, mean, stddev

ree_wasm_export.to_csv('../logs/speedtest1/ree_wasm_formatted.csv', float_format='%.2f')
tee_wasm_export.to_csv('../logs/speedtest1/tee_wasm_formatted.csv', float_format='%.2f')
tee_native_export.to_csv('../logs/speedtest1/tee_native_formatted.csv', float_format='%.2f')

In [ ]:
# Export LaTeX

prefix = "SpeedtestOne"

def write(file, column_name, number, digits_round = 3):
    f.write(f"\\def\\bm{prefix}{column_name}{{{formatForLatex(number, digits_round)}}}\n")

f = open('../logs/speedtest1/export.tex', "w")
write(f, "GlobalSlowdownTeeWasm", tee_wasm_mean_of_mean, 2)
write(f, "GlobalSlowdownTeeNative", tee_native_mean_of_mean, 2)
write(f, "GlobalSlowdownReeWasm", ree_wasm_mean_of_mean, 2)
write(f, "GlobalRatioTeeWasmAndTeeNative", tee_wasm_and_tee_native_ratio, 2)
write(f, "InsertAverage", tee_wasm_insert_mean.values[0], 2)
write(f, "IndexAverage", tee_wasm_index_mean.values[0], 2)
write(f, "SelectAverage", tee_wasm_select_mean.values[0], 2)
write(f, "GlobalSlowdownPercent", (tee_wasm_mean_of_mean - 1) * 100, 1)
write(f, "InsertAveragePercent", (tee_wasm_insert_mean.values[0] - 1) * 100, 1)
write(f, "IndexAveragePercent", (tee_wasm_index_mean.values[0] - 1) * 100, 1)
write(f, "SelectAveragePercent", (tee_wasm_select_mean.values[0] - 1) * 100, 1)
f.close()